
Italy Energy System Optimization - Brownfield Analysis with ETS1 and ETS2
==========================================================================

This script creates an energy system optimization model for Italy as a whole system,
studying the effects of ETS1 (EU Emissions Trading System) and ETS2 (new ETS for 
buildings and transport) on the optimal design and operation of energy technologies.

The analysis includes:
1. Baseline optimization without ETS
2. Optimization with ETS1 only
3. Optimization with both ETS1 and ETS2
4. Comparative analysis of results

In [1]:
import adopt_net0 as adopt
import json
import pandas as pd
from pathlib import Path
import numpy as np

# Create folder for results
results_data_path = Path("./italy_results")
results_data_path.mkdir(parents=True, exist_ok=True)
# Create input data path and optimization templates
input_data_path_baseline = Path("./italy_baseline")
input_data_path_baseline.mkdir(parents=True, exist_ok=True)
adopt.create_optimization_templates(input_data_path_baseline)

Files already exist: italy_baseline\Topology.json italy_baseline\ConfigModel.json


In [3]:
# Load json template
with open(input_data_path_baseline / "Topology.json", "r") as json_file:
    topology = json.load(json_file)
# Nodes
topology["nodes"] = ["Northwest", "Northeast", "Center", "South", "Islands"]
# Carriers:
topology["carriers"] = ["electricity", "heat", "gas", "hydrogen"]
# Investment periods:
topology["investment_periods"] = ["period1", "period2", "period3"]
# Save json template
with open(input_data_path_baseline / "Topology.json", "w") as json_file:
    json.dump(topology, json_file, indent=4)

In [4]:
# Load json template
with open(input_data_path_baseline / "ConfigModel.json", "r") as json_file:
    configuration = json.load(json_file)
# Change objective
configuration["optimization"]["objective"]["value"] = "costs"
# Set MILP gap
configuration["solveroptions"]["mipgap"]["value"] = 0.02
# Save json template
with open(input_data_path_baseline / "ConfigModel.json", "w") as json_file:
    json.dump(configuration, json_file, indent=4)

In [5]:
adopt.create_input_data_folder_template(input_data_path_baseline)

# Add technologies for each period
for period in ["period1", "period2", "period3"]:
    for node in  ["Northwest", "Northeast", "Center", "South", "Islands"]:
        tech_path = input_data_path_baseline / period / "node_data" / node / "Technologies.json"
        
        # Regional scaling factors based on Italian macro regions
        if node == "Northwest":  # Liguria, Lombardy, Piedmont, Valle d'Aosta
            scale = 0.35 # Largest industrial region
        elif node == "Northeast": # Emilia-Romagna, Friuli, Trentino, Veneto
            scale = 0.25  # Strong industrial and agricultural region
        elif node == "Center":  # Lazio, Marche, Tuscany, Umbria
            scale = 0.20  # Central Italy with Rome
        elif node == "South":  # Abruzzo, Basilicata, Calabria, Campania, Molise, Puglia
            scale = 0.15  # Southern mainland Italy
        else:  # Islands
            scale = 0.05  # Islands

        technologies = {
            "new": ["Photovoltaic", "HeatPump_AirSourced", "Storage_Battery", 
                   "WindTurbine_Onshore_4000", "Storage_H2", "WindTurbine_Offshore_9500"],
            "existing": {
                "GasTurbine_simple": int(20000 * scale),
                "Boiler_Small_NG": int(15000 * scale)
            }
        }
        
        with open(tech_path, "w") as json_file:
            json.dump(technologies, json_file, indent=4)

# Copy over technology files
adopt.copy_technology_data(input_data_path_baseline)

# Specify location of the households
# Specify locations
node_location = pd.read_csv(input_data_path_baseline / "NodeLocations.csv", sep=';', index_col=0, header=0)
locations = {
    'Northwest': [8.95, 45.47, 250],    # Milan/Turin area (industrial northwest)
    'Northeast': [11.88, 45.41, 150],   # Venice/Bologna area (northeast)
    'Center': [12.49, 42.35, 200],      # Rome/Florence area (central Italy)
    'South': [15.55, 40.64, 150],       # Naples/Bari area (southern Italy)
    'Islands': [13.36, 38.12, 200]      # Palermo/Cagliari area (Sicily/Sardinia)
}

for node, coords in locations.items():
    node_location.at[node, 'lon'] = coords[0]
    node_location.at[node, 'lat'] = coords[1]
    node_location.at[node, 'alt'] = coords[2]

node_location = node_location.reset_index()
node_location.to_csv(input_data_path_baseline / "NodeLocations.csv", sep=';', index=False)

==================================================================================

In [6]:
# Load demand data - scale household data to Italy level
household_data = adopt.load_household_data()

# Italy annual demands (TWh scaled to MW profiles)
italy_electricity_scale = 320000 / (household_data.iloc[:, 1].sum() / 1000)  # Scale to 320 TWh
italy_heat_scale = 200000 / (household_data.iloc[:, 0].sum() / 1000)  # Scale to 200 TWh


for period in ["period1", "period2", "period3"]:
    for node in ["Northwest", "Northeast", "Center", "South", "Islands"]:
        # Regional scaling
        if node == "Northwest":
            regional_scale = 0.35
        elif node == "Northeast":
            regional_scale = 0.25
        elif node == "Center":
            regional_scale = 0.20
        elif node == "South":
            regional_scale = 0.15
        else:  # Islands
            regional_scale = 0.05
        
        # Create demand profiles
        el_demand = household_data.iloc[:, 1] * italy_electricity_scale * regional_scale
        heat_demand = household_data.iloc[:, 0] * italy_heat_scale * regional_scale
        
        # Fill demand data
        adopt.fill_carrier_data(input_data_path_baseline, value_or_data=el_demand, columns=['Demand'], carriers=['electricity'], nodes=[node])
        adopt.fill_carrier_data(input_data_path_baseline, value_or_data=heat_demand, columns=['Demand'], carriers=['heat'], nodes=[node])

In [7]:
# Set baseline energy prices (no carbon cost)
for period in ["period1", "period2", "period3"]:
    for node in ["North_Italy", "Central_Italy", "South_Italy", "Sardinia"]:
        adopt.fill_carrier_data(input_data_path_baseline, value_or_data=80, columns=['Import price'], carriers=['gas'], nodes=[node])
        adopt.fill_carrier_data(input_data_path_baseline, value_or_data=100, columns=['Import price'], carriers=['electricity'], nodes=[node])
        adopt.fill_carrier_data(input_data_path_baseline, value_or_data=10000, columns=['Import limit'], carriers=['gas'], nodes=[node])

In [8]:
# Set carbon cost to zero (baseline)
for period in ["period1", "period2", "period3"]:
    for node in ["Northwest", "Northeast", "Center", "South", "Islands"]:
        carbon_cost_path = input_data_path_baseline / period / "node_data" / node / "CarbonCost.csv"
        if carbon_cost_path.exists():
            carbon_cost = pd.read_csv(carbon_cost_path, sep=';', index_col=0, header=0)
            carbon_cost['price'] = np.zeros(8760)
            carbon_cost = carbon_cost.reset_index()
            carbon_cost.to_csv(carbon_cost_path, sep=';', index=False)


In [8]:
m = adopt.ModelHub()
m.read_data(input_data_path_baseline)
m.quick_solve()

--- Reading in data ---
Input data folder has been checked successfully - no errors occurred.
Reading data from italy_baseline
Topology read successfully
Model Configuration read successfully
Time series read successfully
Node Locations read successfully
Energy balance options read successfully
Deriving performance data for Heat Pump...


Complete:  99.0 %99999999999 %%Complete:  100 %


Deriving performance data for Heat Pump...


Complete:  99.0 %Complete:  100 %


Deriving performance data for Heat Pump...


...

Complete:  99.0 %99999999999 %%Complete:  100 %


Deriving performance data for Heat Pump...


Complete:  99.0 %Complete:  100 %


Technology data read successfully
Network data read successfully
--- Reading in data complete ---
--- Constructing Model ---
Set parameter Username
Set parameter LicenseID to value 2671115
Academic license - for non-commercial use only - expires 2026-05-26
Constructing Investment Period period1
Constructing Investment Period period1 completed
	 - Adding Technology Photovoltaic
DEPRECATED: Using __getitem__ to return a set value from its (ordered)
position is deprecated.  Please use at()  (deprecated in 6.1, will be
removed in (or after) 7.0)
(called from c:\Users\Jamme002\OneDrive - Universiteit Utrecht\Documents\YOU\adopt_net0\components\technologies\technology.py:931)
	 - Adding Technology HeatPump_AirSourced
	 - Adding Technology Storage_Battery
			gdp.bigm Transformation...
			gdp.bigm Transformation completed in 17 s
	 - Adding Technology WindTurbine_Onshore_4000
	 - Adding Technology Storage_H2
			gdp.bigm Transformation...
			gdp.bigm Transformation completed in 15 s
	 - Adding 

MemoryError: 

In [8]:
m = adopt.ModelHub()
m.read_data(input_data_path_baseline)
result_baseline = m.quick_solve()
print("✓ Baseline scenario completed")

--- Reading in data ---
Input data folder has been checked successfully - no errors occurred.
Reading data from italy_baseline
Topology read successfully
Model Configuration read successfully
Time series read successfully
Node Locations read successfully
Energy balance options read successfully
Deriving performance data for Heat Pump...


Complete:  99.0 %Complete:  100 %


Deriving performance data for Heat Pump...


Complete:  99.0 %Complete:  100 %


Deriving performance data for Heat Pump...


...

Complete:  99.0 %Complete:  100 %


Deriving performance data for Heat Pump...


Complete:  99.0 %Complete:  100 %


Technology data read successfully
Network data read successfully
--- Reading in data complete ---
--- Constructing Model ---
Set parameter Username
Set parameter LicenseID to value 2671115
Academic license - for non-commercial use only - expires 2026-05-26
Constructing Investment Period period1
Constructing Investment Period period1 completed
	 - Adding Technology Photovoltaic
DEPRECATED: Using __getitem__ to return a set value from its (ordered)
position is deprecated.  Please use at()  (deprecated in 6.1, will be
removed in (or after) 7.0)
(called from c:\Users\Jamme002\OneDrive - Universiteit Utrecht\Documents\YOU\adopt_net0\components\technologies\technology.py:931)
	 - Adding Technology HeatPump_AirSourced
	 - Adding Technology Storage_Battery
			gdp.bigm Transformation...
			gdp.bigm Transformation completed in 16 s
	 - Adding Technology WindTurbine_Onshore_4000
	 - Adding Technology Storage_H2
			gdp.bigm Transformation...
			gdp.bigm Transformation completed in 12 s
	 - Adding 

MemoryError: 

===============================================================================================================================

In [ ]:
## Scenario 2: ETS1 Only
# Create ETS1 scenario by copying baseline and modifying carbon costs
input_data_path_ets1 = Path("./italy_ets1")
import shutil
shutil.copytree(input_data_path_baseline, input_data_path_ets1, dirs_exist_ok=True)

# Set ETS1 carbon prices (EUR/t CO2)
ets1_prices = {"period1": 85, "period2": 120, "period3": 160}

In [ ]:

for period in ["period1", "period2", "period3"]:
    for node in ["Northwest", "Northeast", "Center", "South", "Islands"]:
        # Update carbon costs for ETS1
        carbon_cost_path = input_data_path_ets1 / period / "node_data" / node / "CarbonCost.csv"
        carbon_cost = pd.read_csv(carbon_cost_path, sep=';', index_col=0, header=0)
        carbon_cost['price'] = np.ones(8760) * ets1_prices[period]
        carbon_cost = carbon_cost.reset_index()
        carbon_cost.to_csv(carbon_cost_path, sep=';', index=False)
        
        # Update gas prices to include ETS1 carbon cost
        gas_carbon_cost = ets1_prices[period] * 0.202  # 0.202 t CO2/MWh for gas
        adopt.fill_carrier_data(input_data_path_ets1, value_or_data=80 + gas_carbon_cost, columns=['Import price'], carriers=['gas'], nodes=[node], periods=[period])


In [ ]:
# Run ETS1 model
m_ets1 = adopt.ModelHub()
m_ets1.read_data(input_data_path_ets1)
result_ets1 = m_ets1.quick_solve()
print("✓ ETS1 scenario completed")

In [ ]:
## Scenario 3: ETS1 + ETS2
# Create ETS1+ETS2 scenario
input_data_path_ets12 = Path("./italy_ets1_ets2")
shutil.copytree(input_data_path_ets1, input_data_path_ets12, dirs_exist_ok=True)


# Set ETS2 additional carbon prices
ets2_additional = {"period1": 30, "period2": 50, "period3": 70}

In [ ]:
for period in ["period1", "period2", "period3"]:
    for node in ["Northwest", "Northeast", "Center", "South", "Islands"]:
        # Update carbon costs for ETS1 + ETS2 (use higher price)
        total_carbon_price = max(ets1_prices[period], ets1_prices[period] + ets2_additional[period])
        
        carbon_cost_path = input_data_path_ets12 / period / "node_data" / node / "CarbonCost.csv"
        carbon_cost = pd.read_csv(carbon_cost_path, sep=';', index_col=0, header=0)
        carbon_cost['price'] = np.ones(8760) * total_carbon_price
        carbon_cost = carbon_cost.reset_index()
        carbon_cost.to_csv(carbon_cost_path, sep=';', index=False)
        
        # Update gas prices for residential heating (ETS2 impact)
        gas_ets2_cost = (ets1_prices[period] + ets2_additional[period]) * 0.202
        adopt.fill_carrier_data(input_data_path_ets12, value_or_data=80 + gas_ets2_cost, columns=['Import price'], carriers=['gas'], nodes=[node], periods=[period])

In [ ]:
# Run ETS1+ETS2 model
m_ets12 = adopt.ModelHub()
m_ets12.read_data(input_data_path_ets12)
result_ets12 = m_ets12.quick_solve()
print("ETS1 + ETS2 scenario completed")

In [ ]:
# Extract key results
scenarios = ["Baseline", "ETS1 Only", "ETS1 + ETS2"]
models = [m, m_ets1, m_ets12]
results = [result_baseline, result_ets1, result_ets12]

In [ ]:
# Create results summary
results_summary = {
    'Scenario': scenarios,
    'Total_Cost_Billion_EUR': [],
    'Status': []
}

for i, (scenario, model, result) in enumerate(zip(scenarios, models, results)):
    print(f"\n{scenario}:")
    if hasattr(result, 'best_objective'):
        cost_billion = result.best_objective / 1e9
        results_summary['Total_Cost_Billion_EUR'].append(round(cost_billion, 2))
        results_summary['Status'].append('Optimal')
        print(f"  Total Cost: {cost_billion:.2f} billion EUR")
    else:
        results_summary['Total_Cost_Billion_EUR'].append(0)
        results_summary['Status'].append('Not solved')
        print(f"  Status: {result}")


In [ ]:
# Save results summary
results_df = pd.DataFrame(results_summary)
results_df.to_csv(results_data_path / "italy_ets_comparison.csv", index=False)
